In [1]:
from datasets import load_from_disk

In [3]:
ds = load_from_disk("pubmed_preproc")

Loading dataset from disk:   0%|          | 0/215 [00:00<?, ?it/s]

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['h-index', 'sjr', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask', 'random'],
        num_rows: 57940603
    })
    validation: Dataset({
        features: ['h-index', 'sjr', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 3049506
    })
})

In [7]:
ds["train"][0]["random"] 

2.3444218515250483